# Importando módulos 

In [15]:
import obspy
from obspy.taup import TauPyModel

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd

#para plotar as figuras
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition,inset_axes
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from datetime import datetime,timedelta,date
from tqdm import tqdm

from shapely.geometry.polygon import LinearRing

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Inputs e Outputs

In [22]:
FOLDER_OUTPUT = '/run/media/dIOGOLOC/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/'

MSEED_INPUT = "/run/media/dIOGOLOC/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/MSEED/"

METADATA_OUTPUT = "/run/media/dIOGOLOC/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/METADATA/"

filename_csv = '/run/media/dIOGOLOC/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/metadados_glider_acustico_pmpas-bs.csv'

# Extraindo informações dos arquivos ".mseed"

In [17]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*.mseed'))

In [18]:
def mseed_data_2_dataframe(i):
    subdir, filename_wav = os.path.split(i)
    filename = filename_wav.split('.mseed')[0]
    if 'pa' in filename.split('_')[0]:
        mergulho = filename.split('_')[0].split('a')[1]
        stream_number = filename.split('_')[1]

        year_month_day = filename.split('_')[2]
        hour_minute_second = filename.split('_')[3]

        year = int('20'+year_month_day[:2])
        month = int(year_month_day[2:4])
        day = int(year_month_day[4:])

        hour = int(hour_minute_second[:2])
        minute = int(hour_minute_second[2:4])
        second = int(hour_minute_second[4:])

        d = UTCDateTime(datetime(year,month,day,hour,minute,second).isoformat())


    if 'pa' in filename.split('_')[2]:

        mergulho = filename.split('_')[2].split('a')[1]
        stream_number = filename.split('_')[3]

        year_month_day = filename.split('_')[0]
        hour_minute_second = filename.split('_')[1]

        year = int('20'+year_month_day[:2])
        month = int(year_month_day[2:4])
        day = int(year_month_day[4:])

        hour = int(hour_minute_second[:2])
        minute = int(hour_minute_second[2:4])
        second = int(hour_minute_second[4:])

        d = UTCDateTime(datetime(year,month,day,hour,minute,second).isoformat())
        
    
    st = read(i,headonly=True)   
    #----------------------------
    #Starting Dataframe

    starttime = st[0].stats.starttime.datetime
    endtime = st[0].stats.endtime.datetime
    sampling_rate = st[0].stats.sampling_rate
    npts = st[0].stats.npts

    
    df = pd.DataFrame([[filename],[mergulho],[stream_number],[starttime],[endtime],[sampling_rate],[npts]], index=['filename_mseed', 'mergulho', 'stream_number','starttime','endtime','sampling_rate','npts']).T
    
    #Ending Dataframe
    #----------------------------
    return df

In [19]:
pandas_mseed_lst = []

with Pool(processes=8) as p:
    max_ = len(filenames_MSEED)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(mseed_data_2_dataframe,filenames_MSEED):
            pbar.update()
            pandas_mseed_lst.append(result)

100%|██████████████████████████████████| 109315/109315 [06:48<00:00, 267.43it/s]


In [20]:
dataframe_mseed_final = pd.concat(pandas_mseed_lst, ignore_index=True)

In [21]:
dataframe_mseed_final.sort_values(by='starttime')

,filename_mseed,mergulho,stream_number,starttime,endtime,sampling_rate,npts
15,pa0001au_001_151112_134841,0001,001,2015-11-12 13:48:41,2015-11-12 13:50:40.930,100.0,11994
30,pa0001au_002_151112_135041,0001,002,2015-11-12 13:50:41,2015-11-12 13:52:40.930,100.0,11994
3,pa0001au_003_151112_135241,0001,003,2015-11-12 13:52:41,2015-11-12 13:54:40.930,100.0,11994
1,pa0001au_004_151112_135441,0001,004,2015-11-12 13:54:41,2015-11-12 13:56:40.930,100.0,11994
20,pa0001au_005_151112_135641,0001,005,2015-11-12 13:56:41,2015-11-12 13:57:00.980,100.0,1999
...,...,...,...,...,...,...,...
109306,pa0238au_013_191010_211037,0238,013,2019-10-10 21:10:37,2019-10-10 21:20:36.990,100.0,60000
109307,pa0238au_014_191010_212037,0238,014,2019-10-10 21:20:37,2019-10-10 21:30:36.990,100.0,60000
109308,pa0238au_015_191010_213037,0238,015,2019-10-10 21:30:37,2019-10-10 21:40:36.990,100.0,60000
109309,pa0238au_016_191010_214037,0238,016,2019-10-10 21:40:37,2019-10-10 21:50:36.990,100.0,60000


# Extraindo informações dos arquivos ".csv"

In [23]:
dataframe_csv = pd.read_csv(filename_csv,parse_dates=['time'])
dataframe_csv.sort_values(by='time')

,time,filename,latitude,longitude,depth
0,2015-11-12 13:48:40.999995,pa0001au_001_151112_134841_rms_spl3.mat,-23.825157,-42.495483,1.314286
1,2015-11-12 13:49:41.000001,pa0001au_001_151112_134841_rms_spl3.mat,-23.824950,-42.495487,7.415555
2,2015-11-12 13:50:40.999998,pa0001au_002_151112_135041_rms_spl3.mat,-23.824745,-42.495490,15.776000
3,2015-11-12 13:51:41.000004,pa0001au_002_151112_135041_rms_spl3.mat,-23.824540,-42.495495,24.372000
4,2015-11-12 13:52:41.000001,pa0001au_003_151112_135241_rms_spl3.mat,-23.824331,-42.495500,32.912000
...,...,...,...,...,...
547748,2021-11-04 06:52:38.000000,pa0296au_011_211104_064338_rms_spl3.mat,-24.461742,-42.471264,866.012900
547749,2021-11-04 06:53:37.999997,pa0296au_012_211104_065338_rms_spl3.mat,-24.461819,-42.471302,873.586100
547750,2021-11-04 06:54:38.000003,pa0296au_012_211104_065338_rms_spl3.mat,-24.461980,-42.471510,879.579600
547751,2021-11-04 06:55:38.000000,pa0296au_012_211104_065338_rms_spl3.mat,-24.462152,-42.471740,884.380700


In [24]:
df_csv = dataframe_csv.groupby("filename").agg(pd.Series.tolist)

In [25]:
df_csv

,time,latitude,longitude,depth
filename,,,,
160708_172008_pa0003au_001_rms_spl3.mat,"[2016-07-08 17:20:07.999999, 2016-07-08 17:21:...","[-24.391026, -24.391314, -24.391335]","[-43.821663, -43.82173, -43.82174]","[0.9987981, 9.891874, 10.969999]"
160708_172307_pa0003au_002_rms_spl3.mat,"[2016-07-08 17:23:07.000004, 2016-07-08 17:24:...","[-24.391356, -24.3916, -24.391617]","[-43.821743, -43.8218, -43.821804]","[12.030156, 23.590548, 24.420235]"
160708_172607_pa0003au_003_rms_spl3.mat,"[2016-07-08 17:26:07.000003, 2016-07-08 17:27:...","[-24.391909, -24.39193, -24.392172]","[-43.821873, -43.82188, -43.821938]","[36.456642, 37.333202, 47.245785]"
160708_172906_pa0003au_004_rms_spl3.mat,"[2016-07-08 17:29:05.999998, 2016-07-08 17:30:...","[-24.392193, -24.392475, -24.392496]","[-43.82194, -43.822006, -43.82201]","[48.01094, 58.52969, 59.289062]"
160708_173205_pa0003au_005_rms_spl3.mat,"[2016-07-08 17:32:05.000003, 2016-07-08 17:33:...","[-24.392744, -24.392765, -24.393057]","[-43.822067, -43.822075, -43.822144]","[68.57156, 69.49031, 80.89149]"
...,...,...,...,...
pa0626au_041_160418_075202_rms_spl3.mat,"[2016-04-18 07:52:02.000001, 2016-04-18 07:53:...","[-24.46632, -24.466305]","[-43.148865, -43.14886]","[897.4117, 899.9383]"
pa0626au_042_160418_075402_rms_spl3.mat,"[2016-04-18 07:54:02.000004, 2016-04-18 07:55:02]","[-24.466183, -24.466167]","[-43.14885, -43.148846]","[918.85736, 921.4589]"
pa0626au_043_160418_075602_rms_spl3.mat,"[2016-04-18 07:56:01.999997, 2016-04-18 07:57:...","[-24.466047, -24.466032]","[-43.148834, -43.14883]","[940.0661, 942.0864]"


In [26]:
df_csv['filename_mseed'] = df_csv.index.str.replace('_rms_spl3.mat', '')

/tmp/ipykernel_244827/1878750020.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_csv['filename_mseed'] = df_csv.index.str.replace('_rms_spl3.mat', '')


In [27]:
df_csv.sort_values(by='time')

,time,latitude,longitude,depth,filename_mseed
filename,,,,,
pa0001au_001_151112_134841_rms_spl3.mat,"[2015-11-12 13:48:40.999995, 2015-11-12 13:49:...","[-23.825157, -23.82495]","[-42.495483, -42.495487]","[1.3142858, 7.4155555]",pa0001au_001_151112_134841
pa0001au_002_151112_135041_rms_spl3.mat,"[2015-11-12 13:50:40.999998, 2015-11-12 13:51:...","[-23.824745, -23.82454]","[-42.49549, -42.495495]","[15.776, 24.372]",pa0001au_002_151112_135041
pa0001au_003_151112_135241_rms_spl3.mat,"[2015-11-12 13:52:41.000001, 2015-11-12 13:53:...","[-23.824331, -23.824125]","[-42.4955, -42.495502]","[32.912, 41.34]",pa0001au_003_151112_135241
pa0001au_004_151112_135441_rms_spl3.mat,"[2015-11-12 13:54:41.000004, 2015-11-12 13:55:41]","[-23.82392, -23.823713]","[-42.495506, -42.495506]","[48.91, 52.824]",pa0001au_004_151112_135441
pa0002au_001_151112_143618_rms_spl3.mat,"[2015-11-12 14:36:17.999998, 2015-11-12 14:37:...","[-23.817575, -23.81744]","[-42.49506, -42.494965]","[1.4671428, 9.36]",pa0002au_001_151112_143618
...,...,...,...,...,...
pa0296au_008_211104_061338_rms_spl3.mat,"[2021-11-04 06:13:37.999999, 2021-11-04 06:14:...","[-24.457632, -24.457657, -24.457726, -24.45786...","[-42.470257, -42.470303, -42.470325, -42.47033...","[550.4096, 559.3073, 567.8462, 575.10706, 582....",pa0296au_008_211104_061338
pa0296au_009_211104_062338_rms_spl3.mat,"[2021-11-04 06:23:38.000003, 2021-11-04 06:24:...","[-24.458628, -24.458672, -24.45872, -24.458822...","[-42.47047, -42.470528, -42.470562, -42.470573...","[629.96436, 637.68353, 646.56323, 654.6224, 66...",pa0296au_009_211104_062338
pa0296au_010_211104_063338_rms_spl3.mat,"[2021-11-04 06:33:37.999998, 2021-11-04 06:34:...","[-24.459415, -24.459513, -24.45957, -24.45963,...","[-42.4708, -42.470802, -42.470856, -42.4709, -...","[706.7263, 715.5506, 722.5956, 730.11316, 737....",pa0296au_010_211104_063338


# Aglutinando os dataframes: 

In [28]:
merged_df_csv_mseed = df_csv.merge(dataframe_mseed_final, on='filename_mseed')

In [29]:
merged_df_csv_mseed.sort_values(by='starttime')

,time,latitude,longitude,depth,filename_mseed,mergulho,stream_number,starttime,endtime,sampling_rate,npts
76390,"[2015-11-12 13:48:40.999995, 2015-11-12 13:49:...","[-23.825157, -23.82495]","[-42.495483, -42.495487]","[1.3142858, 7.4155555]",pa0001au_001_151112_134841,0001,001,2015-11-12 13:48:41,2015-11-12 13:50:40.930,100.0,11994
76394,"[2015-11-12 13:50:40.999998, 2015-11-12 13:51:...","[-23.824745, -23.82454]","[-42.49549, -42.495495]","[15.776, 24.372]",pa0001au_002_151112_135041,0001,002,2015-11-12 13:50:41,2015-11-12 13:52:40.930,100.0,11994
76398,"[2015-11-12 13:52:41.000001, 2015-11-12 13:53:...","[-23.824331, -23.824125]","[-42.4955, -42.495502]","[32.912, 41.34]",pa0001au_003_151112_135241,0001,003,2015-11-12 13:52:41,2015-11-12 13:54:40.930,100.0,11994
76402,"[2015-11-12 13:54:41.000004, 2015-11-12 13:55:41]","[-23.82392, -23.823713]","[-42.495506, -42.495506]","[48.91, 52.824]",pa0001au_004_151112_135441,0001,004,2015-11-12 13:54:41,2015-11-12 13:56:40.930,100.0,11994
76428,"[2015-11-12 14:36:17.999998, 2015-11-12 14:37:...","[-23.817575, -23.81744]","[-42.49506, -42.494965]","[1.4671428, 9.36]",pa0002au_001_151112_143618,0002,001,2015-11-12 14:36:18,2015-11-12 14:38:17.930,100.0,11994
...,...,...,...,...,...,...,...,...,...,...,...
96568,"[2019-10-10 21:10:37.000005, 2019-10-10 21:11:...","[-24.700645, -24.700645, -24.700645, -24.70064...","[-41.329514, -41.329514, -41.329514, -41.32951...","[810.5255, 810.5255, 810.5255, 810.5255, 810.5...",pa0238au_013_191010_211037,0238,013,2019-10-10 21:10:37,2019-10-10 21:20:36.990,100.0,60000
96570,"[2019-10-10 21:20:36.999999, 2019-10-10 21:21:...","[-24.700645, -24.700645, -24.700645, -24.70064...","[-41.329514, -41.329514, -41.329514, -41.32951...","[810.5255, 810.5255, 810.5255, 810.5255, 810.5...",pa0238au_014_191010_212037,0238,014,2019-10-10 21:20:37,2019-10-10 21:30:36.990,100.0,60000
96572,"[2019-10-10 21:30:37.000004, 2019-10-10 21:31:...","[-24.700645, -24.700645, -24.700645, -24.70064...","[-41.329514, -41.329514, -41.329514, -41.32951...","[810.5255, 810.5255, 810.5255, 810.5255, 810.5...",pa0238au_015_191010_213037,0238,015,2019-10-10 21:30:37,2019-10-10 21:40:36.990,100.0,60000
96574,"[2019-10-10 21:40:36.999998, 2019-10-10 21:41:...","[-24.700645, -24.700645, -24.700645, -24.70064...","[-41.329514, -41.329514, -41.329514, -41.32951...","[810.5255, 810.5255, 810.5255, 810.5255, 810.5...",pa0238au_016_191010_214037,0238,016,2019-10-10 21:40:37,2019-10-10 21:50:36.990,100.0,60000


In [30]:
# Definir uma função para converter a lista de tempos em timestamps
def converter_para_timestamp(tempos):
    return [pd.to_datetime(tempo).timestamp() for tempo in tempos]

In [31]:
merged_df_csv_mseed['time_timestamp'] = merged_df_csv_mseed['time'].apply(lambda x: converter_para_timestamp(x))

In [32]:
merged_df_csv_mseed

,time,latitude,longitude,depth,filename_mseed,mergulho,stream_number,starttime,endtime,sampling_rate,npts,time_timestamp
0,"[2016-07-08 17:20:07.999999, 2016-07-08 17:21:...","[-24.391026, -24.391314, -24.391335]","[-43.821663, -43.82173, -43.82174]","[0.9987981, 9.891874, 10.969999]",160708_172008_pa0003au_001,0003,001,2016-07-08 17:20:08,2016-07-08 17:23:07.300,100.0,17931,"[1467998407.999999, 1467998467.999995, 1467998..."
1,"[2016-07-08 17:23:07.000004, 2016-07-08 17:24:...","[-24.391356, -24.3916, -24.391617]","[-43.821743, -43.8218, -43.821804]","[12.030156, 23.590548, 24.420235]",160708_172307_pa0003au_002,0003,002,2016-07-08 17:23:07,2016-07-08 17:26:06.300,100.0,17931,"[1467998587.000004, 1467998647.0, 1467998706.9..."
2,"[2016-07-08 17:26:07.000003, 2016-07-08 17:27:...","[-24.391909, -24.39193, -24.392172]","[-43.821873, -43.82188, -43.821938]","[36.456642, 37.333202, 47.245785]",160708_172607_pa0003au_003,0003,003,2016-07-08 17:26:07,2016-07-08 17:29:06.300,100.0,17931,"[1467998767.000003, 1467998827.0, 1467998886.9..."
3,"[2016-07-08 17:29:05.999998, 2016-07-08 17:30:...","[-24.392193, -24.392475, -24.392496]","[-43.82194, -43.822006, -43.82201]","[48.01094, 58.52969, 59.289062]",160708_172906_pa0003au_004,0003,004,2016-07-08 17:29:06,2016-07-08 17:32:05.300,100.0,17931,"[1467998945.999998, 1467999006.000004, 1467999..."
4,"[2016-07-08 17:32:05.000003, 2016-07-08 17:33:...","[-24.392744, -24.392765, -24.393057]","[-43.822067, -43.822075, -43.822144]","[68.57156, 69.49031, 80.89149]",160708_173205_pa0003au_005,0003,005,2016-07-08 17:32:05,2016-07-08 17:35:04.300,100.0,17931,"[1467999125.000003, 1467999184.999999, 1467999..."
...,...,...,...,...,...,...,...,...,...,...,...,...
105394,"[2016-04-18 07:52:02.000001, 2016-04-18 07:53:...","[-24.46632, -24.466305]","[-43.148865, -43.14886]","[897.4117, 899.9383]",pa0626au_041_160418_075202,0626,041,2016-04-18 07:52:02,2016-04-18 07:54:01.930,100.0,11994,"[1460965922.000001, 1460965981.999997]"
105395,"[2016-04-18 07:54:02.000004, 2016-04-18 07:55:02]","[-24.466183, -24.466167]","[-43.14885, -43.148846]","[918.85736, 921.4589]",pa0626au_042_160418_075402,0626,042,2016-04-18 07:54:02,2016-04-18 07:56:01.930,100.0,11994,"[1460966042.000004, 1460966102.0]"
105396,"[2016-04-18 07:56:01.999997, 2016-04-18 07:57:...","[-24.466047, -24.466032]","[-43.148834, -43.14883]","[940.0661, 942.0864]",pa0626au_043_160418_075602,0626,043,2016-04-18 07:56:02,2016-04-18 07:58:01.930,100.0,11994,"[1460966161.999997, 1460966222.000003]"
105397,"[2016-04-18 07:58:02, 2016-04-18 07:59:01.999996]","[-24.465881, -24.465864]","[-43.148815, -43.14881]","[958.9343, 959.4406]",pa0626au_044_160418_075802,0626,044,2016-04-18 07:58:02,2016-04-18 08:00:01.930,100.0,11994,"[1460966282.0, 1460966341.999996]"


In [33]:
def interp_data(df,nfile):
    
    df = df[df['filename_mseed'] == nfile]
    mseed_nfile = df['filename_mseed'].values[0]
    name_file = sorted(glob.glob(MSEED_INPUT+'*/*/'+mseed_nfile+'*'))[0]
    
    st = read(name_file,headonly=True)  
    starttime = st[0].stats.starttime.timestamp
    endtime = st[0].stats.endtime.timestamp
    sampling_rate = st[0].stats.sampling_rate
    npts = st[0].stats.npts
    times = np.array([i.timestamp for i in st[0].times('utcdatetime')])
    times_datetime = np.array([i.datetime for i in st[0].times('utcdatetime')])

    # Dados de profundidade e tempo
    profundidade_lst = df['depth'].values[0]
    latitude_lst = df['latitude'].values[0]
    longitude_lst = df['longitude'].values[0]
    tempo_numerico_lst = df['time_timestamp'].values[0]
    tempo_interpolado_numerico = times
    
    # Interpolar os dados de profundidade usando numpy.interp
    profundidade_interpolada = np.interp(tempo_interpolado_numerico, tempo_numerico_lst, profundidade_lst)

    # Interpolar os dados de longitude usando numpy.interp
    longitude_interpolada = np.interp(tempo_interpolado_numerico, tempo_numerico_lst, longitude_lst)

    # Interpolar os dados de latitude usando numpy.interp
    latitude_interpolada = np.interp(tempo_interpolado_numerico, tempo_numerico_lst, latitude_lst)

    # Exibir os resultados
    df_interpolado = pd.DataFrame({'Datafile':nfile,'Tempo': times_datetime, 'Profundidade_interp': profundidade_interpolada, 'Longitude_interp': longitude_interpolada, 'Latitude_interp': latitude_interpolada})

    #Criando pasta para salvar os metadados 
    folder_feather_name = METADATA_OUTPUT+st[0].stats.starttime.strftime('%Y')+'/'+st[0].stats.starttime.strftime('%Y-%m-%d')+'/'
    os.makedirs(folder_feather_name,exist_ok=True)
    df_interpolado.to_feather(folder_feather_name+mseed_nfile+'.feather')
    
    return df_interpolado

In [34]:
def start_interpol(namfile):
    df_interpol = interp_data(df=merged_df_csv_mseed,nfile=namfile)
    
    return df_interpol

In [ ]:
pandas_mseed_lst = []

with Pool(processes=16) as p:
    max_ = len(merged_df_csv_mseed['filename_mseed'].values)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(start_interpol,merged_df_csv_mseed['filename_mseed'].values):
            pbar.update()
            pandas_mseed_lst.append(result)

 47%|███████████████                 | 49694/105399 [1:32:34<4:21:54,  3.54it/s]

# Importando metadados:

In [ ]:
filenames_metadata = sorted(glob.glob(METADATA_OUTPUT+'*/*/*.feather'))

In [ ]:
for nfile_meta in tqdm(filenames_metadata,total=len(filenames_metadata)):

    #########################################################################################################################################################
    #STREAM
    file_n_meta = nfile_meta.split('/')[-1].split('.feather')[0]
    
    name_file = sorted(glob.glob(MSEED_INPUT+'*/*/'+file_n_meta+'*'))[0]
    
    st = read(name_file,headonly=False)  
    starttime = st[0].stats.starttime
    endtime = st[0].stats.endtime
    sampling_rate = st[0].stats.sampling_rate
    npts = st[0].stats.npts
    
    df_meta = pd.read_feather(nfile_meta)

    lat_mean_info = df_meta['Latitude_interp'].mean()
    lon_mean_info = df_meta['Longitude_interp'].mean()       
    depth_mean_info = df_meta['Profundidade_interp'].mean()

    lat_std_info = df_meta['Latitude_interp'].std()
    lon_std_info = df_meta['Longitude_interp'].std()       
    depth_std_info = df_meta['Profundidade_interp'].std()
    
    
    if 'pa' in df_meta['Datafile'][0].split('_')[0]:
        mergulho = df_meta['Datafile'][0].split('_')[0].split('a')[1]
        stream_number = df_meta['Datafile'][0].split('_')[1]

    if 'pa' in df_meta['Datafile'][0].split('_')[2]:
        mergulho = df_meta['Datafile'][0].split('_')[2].split('a')[1]
        stream_number = df_meta['Datafile'][0].split('_')[3]
                        

In [ ]:
for nfile_meta in tqdm(filenames_metadata,total=len(filenames_metadata)):

    #########################################################################################################################################################
    #STREAM
    file_n_meta = nfile_meta.split('/')[-1].split('.feather')[0]
    
    name_file = sorted(glob.glob(MSEED_INPUT+'*/*/'+file_n_meta+'*'))[0]
    
    st = read(name_file,headonly=False)  
    starttime = st[0].stats.starttime
    endtime = st[0].stats.endtime
    sampling_rate = st[0].stats.sampling_rate
    npts = st[0].stats.npts
    
    df_meta = pd.read_feather(nfile_meta)

    lat_mean_info = df_meta['Latitude_interp'].mean()
    lon_mean_info = df_meta['Longitude_interp'].mean()       
    depth_mean_info = df_meta['Profundidade_interp'].mean()

    lat_std_info = df_meta['Latitude_interp'].std()
    lon_std_info = df_meta['Longitude_interp'].std()       
    depth_std_info = df_meta['Profundidade_interp'].std()
    
    
    if 'pa' in df_meta['Datafile'][0].split('_')[0]:
        mergulho = df_meta['Datafile'][0].split('_')[0].split('a')[1]
        stream_number = df_meta['Datafile'][0].split('_')[1]

    if 'pa' in df_meta['Datafile'][0].split('_')[2]:
        mergulho = df_meta['Datafile'][0].split('_')[2].split('a')[1]
        stream_number = df_meta['Datafile'][0].split('_')[3]
                        
    #########################################################################################################################################################
    #Figure 

    # set up the plot and create a GeoAxes:
    proj = ccrs.PlateCarree()

    fig, (ax,ax1) = plt.subplots(2,1,figsize=(16,16))
    ax = plt.subplot(2, 1, 1, projection=proj)
                     
    # ----------------------------------------------------------------------------------------------------------
    # Limit the extent of the map to a small longitude/latitude range.
    latmin=-27
    latmax=-22
    lonmin=-50
    lonmax=-40

    ax.set_extent([lonmin,lonmax, latmin, latmax], crs=ccrs.Geodetic())

    # ----------------------------------------------------------------------------------------------------------
    # Ploting lat/lon values
                
    h = ax.scatter(df_meta['Longitude_interp'].values,df_meta['Latitude_interp'].values,c=np.array([mdates.date2num(i) for i in df_meta['Tempo'].dt.to_pydatetime()]),marker='o',alpha=0.8,cmap='plasma',s=75,transform=proj)
    ax.text(0.85, 0.05,'Lat:'+str(round(lat_mean_info,2))+r'$\pm$'+str(round(lat_std_info,4)), horizontalalignment='left',verticalalignment='center', transform=ax.transAxes)
    ax.text(0.85, 0.1,'Lon:'+str(round(lon_mean_info,2))+r'$\pm$'+str(round(lon_std_info,4)), horizontalalignment='left',verticalalignment='center', transform=ax.transAxes)
    ax.text(0.85, 0.15,'Prof:'+str(round(depth_mean_info,2))+r'$\pm$'+str(round(depth_std_info,4)), horizontalalignment='left',verticalalignment='center', transform=ax.transAxes)
    # ----------------------------------------------------------------------------------------------------------
    # Adding background map 
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
    ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
    ax.set_title('Dia: '+starttime.strftime('%d-%m-%Y'))
    
    # ----------------------------------------------------------------------------------------------------------
    # Adding colorbar
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size="1%", pad=0.6, axes_class=plt.Axes)

    fig.add_axes(ax_cb)
    cb = plt.colorbar(h, cax=ax_cb)
    cb.ax.yaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    #####################################################
    # Adding inset axes RIGHT
    #####################################################

    axins = plt.axes([0.2, 0.55, 0.2, 0.2],projection=proj)

    axins.scatter(df_meta['Longitude_interp'].values,df_meta['Latitude_interp'].values,c=np.array([mdates.date2num(i) for i in df_meta['Tempo'].dt.to_pydatetime()]),marker='o',alpha=0.3,cmap='plasma',s=75,transform=proj)

    axins.add_feature(cfeature.LAND)
    axins.add_feature(cfeature.OCEAN)
    axins.add_feature(cfeature.COASTLINE,linewidth=0.3)
    axins.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
    
    # ----------------------------------------------------------------------------------------------------------
    # Adding grid 
    axins.gridlines(crs=ccrs.PlateCarree(), draw_labels=["bottom", "left", "right"],linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    axins.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=True, labeltop=False, labelleft=True, labelright=True)

    # subregion of the original image
    x1 = np.min(lon_mean_info)-abs(np.min(lon_mean_info)/1000)
    x2 = np.max(lon_mean_info)+abs(np.max(lon_mean_info)/1000)
    y1 = np.min(lat_mean_info)-abs(np.min(lat_mean_info)/1000)
    y2 = np.max(lat_mean_info)+abs(np.max(lat_mean_info)/1000)
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)

    # ----------------------------------------------------------------------------------------------------------
    # Adding ZOOM
    ax.indicate_inset_zoom(axins, edgecolor="black")

    # ----------------------------------------------------------------------------------------------------------
    ax_histx = plt.axes([0.2, 0.73, 0.2, 0.1],sharex=axins,facecolor='lightsteelblue')
    ax_histx.scatter(df_meta['Longitude_interp'].values,df_meta['Profundidade_interp'].values*-1,c=np.array([mdates.date2num(i) for i in df_meta['Tempo'].dt.to_pydatetime()]),marker='o',alpha=0.7,cmap='plasma',s=100)
    ax_histx.grid(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    ax_histx.set_title("Mergulho:"+mergulho+"\n stream:"+stream_number)
    ax_histx.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=False, labeltop=False, labelleft=True, labelright=True)
    
    depth_lim_max = (np.min(df_meta['Profundidade_interp'])-abs(np.min(df_meta['Profundidade_interp'])/3000))*-1
    depth_lim_min = (np.max(df_meta['Profundidade_interp'])+abs(np.max(df_meta['Profundidade_interp'])/3000))*-1
    ax_histx.set_ylim(depth_lim_min,depth_lim_max)
    
    
    ax_histx.yaxis.set_major_formatter('{x} m')
    ax_histx.yaxis.set_major_locator(MultipleLocator(10))
    ax_histx.yaxis.set_minor_locator(MultipleLocator(1))
    
    # ----------------------------------------------------------------------------------------------------------
    # Adding global location map
    # inset location relative to main plot (ax) in normalized units
    inset_x = 0
    inset_y = 1
    inset_size = 0.5

    # Adding Geoaxes
    ax2 = plt.axes([0, 0, 1, 1], projection=ccrs.Orthographic(central_latitude=(latmin + latmax)/2,central_longitude=(lonmin + lonmax) / 2))
    ax2.set_global()

    # Adding background map 
    ax2.add_feature(cfeature.LAND)
    ax2.add_feature(cfeature.OCEAN)
    ax2.add_feature(cfeature.COASTLINE)

    # Adding inset geoaxes position
    ip = InsetPosition(ax, [inset_x - inset_size / 2,
                          inset_y - inset_size / 2,
                          inset_size,inset_size])
    
    ax2.set_axes_locator(ip)


    # Adding red rectangle position
    nvert = 100
    lons = np.r_[np.linspace(lonmin, lonmin, nvert),
                                 np.linspace(lonmin, lonmax, nvert),
                                 np.linspace(lonmax, lonmax, nvert)].tolist()
    
    lats = np.r_[np.linspace(latmin, latmax, nvert),
                                 np.linspace(latmax, latmax, nvert),
                                 np.linspace(latmax, latmin, nvert)].tolist()

    ring = LinearRing(list(zip(lons, lats)))
    ax2.add_geometries([ring], ccrs.PlateCarree(),facecolor='none', edgecolor='red', linewidth=0.75)

    # ----------------------------------------------------------------------------------------------------------
    # Adding stream plot

    ax1.plot(st[0].times('matplotlib'),st[0].data,'-k')        
    ax1.set_ylabel('Amplitude [counts]')
    
    locator = mdates.AutoDateLocator(minticks=9, maxticks=14)
    formatter = mdates.ConciseDateFormatter(locator)
    
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    
    mins2 = SecondLocator(interval=60)
    mins1 = SecondLocator(interval=10)
    
    ax1.xaxis.set_major_locator(mins2)
    ax1.xaxis.set_minor_locator(mins1)
    ax1.set_title('Arquivo: '+file_n_meta+'.wav')
        
    # ----------------------------------------------------------------------------------------------------------
    # Saving figure
    os.makedirs(FOLDER_OUTPUT+'FIGURAS/INTERP_MSEED/'+st[0].stats.starttime.strftime('%Y')+'/'+st[0].stats.starttime.strftime('%Y-%m-%d')+'/',exist_ok=True)
    fig.savefig(FOLDER_OUTPUT+'FIGURAS/INTERP_MSEED/'+st[0].stats.starttime.strftime('%Y')+'/'+st[0].stats.starttime.strftime('%Y-%m-%d')+'/'+'interp_mseed_'+file_n_meta+'.png')
    plt.close()